<h1>Trie (Keyword Tree)</h1>

A Trie or Keyword tree is a search tree, usually used to search strings. Every child node of a given node in this tree share the same prefix up to that node. Common applications for this data structure include genome analysis, search autocomplete and data analysis. Here we implement a basic trie in C++11 capable of doing insertions and searches.

<h3>How does a Trie works?</h3>

Tries are usually used on data that can be represented in a string format, as a mean to simplificate the explanation we will only be dealing with strings here. Each node on a Trie represents a charachter on the string, the nodes are stored in a top to bottom approach where it's ancestors represent it's prefix. We can see in Figure 1 a Trie after adding the word "Ball".

![Figure 1: Trie after inserting the word "Ball".](Trie1.png)
<b>Figure 1:</b> Trie after inserting the word "Ball".

As shown in the tree in Figure 1, in the word "Ball", the node representing "a" in the seccond layer is predecessed by "B", similarly the first "l" has nodes "Ba" as it's ancestors. The trie allow us to interpret the string as a graph and store the data more efficiently by writting only once the common preffix the strings stored, we can see it on the tree in figure 2 where the words "sweet", "sheep" and "ship were inserted.

<center>
<img src="Trie2.png"/>
<b>Figure 2:</b> Trie after inserting the words "sweet", "sheep" and "ship"
</center>

<h3>Including necessary libraries</h3>

We will represent every chilld node of a given node by a hash and use the node's key as the hash's key which allows for O(1) lookups. I'm personally not a fan of typing "std::" all the damm time and this is a small project so let's use namespace std. The beggining of the file should look something like this.

In [ ]:
#include <unordered_map>

using namespace std;

<h3>The Node</h3>

Each node will have a children Hash which will map a character in that position to it's following nodes. It is also usefull to know if our node represent the end of a word and if it's a leaf node. We also created a default constructor so that the values are preset once the node is initialized. With this in mind our structure must look something like this.

In [ ]:
struct Node{
	unordered_map<char, Node*> children;
	bool endOfWord;
	bool isLeaf;
	Node(){
		endOfWord = false;
		isLeaf = true;
	}
};

<h3>The Trie class</h3>

We chose to represent our Trie as a class so that we can implement good programming practices such as keeping private methods private. The root Node is represented as a pointer to a node and is allocated once the object is initialized, as said before, we also implemented the insert search method.

Since we are allocating the Node pointers as a tree we will have to writte our own destructor. The destructor should be able to visit every node in the tree deallocating the memory. We can achieve that by visiting all child nodes recursively and dealocating them as once all children nodes were visited. This is done by calling the remove method on the destructor which calls itself recursively.

By now, our class is looking like the code bellow:

In [ ]:
class Trie{
private:
    Node *root;
    void remove(Node *n);
public:
    Trie();
    ~Trie();
    
    void insert(string str);
    bool search(string str);
};


In [ ]:
Trie::Trie(){
    root = new Node();
}

<h6>The destructor</h6>

The destructor only calls the remove method passing the root node as a paramter, the remove method free the memory recursively as described earlier. Both, the destructor and remove are shown in the code bellow.

In [ ]:
Trie::~Trie(){
    remove(root);
}

In [ ]:
void Trie::remove(Node *n){
    // Loop thru every child node visiting it recursivelly
    for(pair<char,Node*> p: n->children)
        remove(p.second);
    // Delete current node
    delete n;
}

<h6>The insert method</h6>

We start by visiting the root node and check if any of it's children match the first character on the string, this can be made in $O(1)$ on average with the unordered set. If the node has the carachter as it's child then you simply visit the child node, otherwise you create that child node an visit it, this process repeats with every single letter on the string until reaching the end of it, after reaching the end, the last node reached is marked as a end of word. When creating a new node the current node is marked as not being a leaf node.

The code for this is presented bellow:

In [ ]:
void Trie::insert(string str){
    Node* node = root; // Current node being visited
    // Loop thru every letter on the String
    for(char c: str){
        // Adds a children node if it doesn't find any with that key
        if(node->children.find(c) == node->children.end()){
            node->children[c] = new Node();
            node->isLeaf = false;
        }
        // Visit next node
        node = node->children[c];
    }
    // Sets the end of the word
    node->endOfWord = true;
}

<h6>The search method</h6>

For searching we can take two approaches, one of them is to search for a complete word and the other is to search for a word prefix. Let's say I add the word "hello" on a empty trie, doing a full word search of the word "hell" in this trie should return false, while if you are matching only the prefix, it should return true. Our first code in this topic implements a search where only the prefix is matched a slight modification was made on the second one to match the entire word.

The works by looping thru every carachter on the string and checking if there is a correspondent child node, visiting the node if it was found and returning false otherwise. The function will also return false in case it reaches a leaf node.f 

<b>Important:</b> if you a reading this on a Jupyter notebook please run only one of the codes. If you are tring to implement both in your code, please implement them with different names.

In [ ]:
bool Trie::search(string str){
    Node *node = root; // Current node
    // Loop thru every character on the string
    for(char c:str){
        // Return false if key is not found or reach a leaf node
        if(node->children.find(c) == node->children.end() || node->isLeaf)
            return false;
        // Visit next node
        node = node->children[c];
    }
    // Once all charachters matched, returns true
    return true;
}

The following code returns true only when the the entire word is matched

In [ ]:
bool Trie::search(string str){
    Node *node = root; // Current node
    // Loop thru every character on the string
    for(char c:str){
        // Return false if key is not found or reach a leaf node
        if(node->children.find(c) == node->children.end() || node->isLeaf)
            return false;
        // Visit next node
        node = node->children[c];
    }
    // Returns true once all charachter matched and you reached the end of a word
    return node->endOfWord; // <-- MODIFIED!!!
}

With that we conclude our basic Trie.